In [40]:
# Import necessary libraries
import ipaddress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


In [3]:
df = pd.read_csv('output.csv')

In [11]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TransactionID            2512 non-null   object 
 1   AccountID                2512 non-null   object 
 2   TransactionAmount        2512 non-null   float64
 3   TransactionDate          2512 non-null   object 
 4   TransactionType          2512 non-null   object 
 5   Location                 2512 non-null   object 
 6   DeviceID                 2512 non-null   object 
 7   IP Address               2512 non-null   object 
 8   MerchantID               2512 non-null   object 
 9   Channel                  2512 non-null   object 
 10  CustomerAge              2512 non-null   int64  
 11  CustomerOccupation       2512 non-null   object 
 12  TransactionDuration      2512 non-null   int64  
 13  LoginAttempts            2512 non-null   int64  
 14  AccountBalance          

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance
count,2512.000000,2512.000000,2512.000000,2512.000000,2512.000000
mean,297.593778,44.673965,119.643312,1.124602,5114.302966
std,291.946243,17.792198,69.963757,0.602662,3900.942499
min,0.260000,18.000000,10.000000,1.000000,101.250000
25%,81.885000,27.000000,63.000000,1.000000,1504.370000
50%,211.140000,45.000000,112.500000,1.000000,4735.510000
75%,414.527500,59.000000,161.000000,1.000000,7678.820000
max,1919.110000,80.000000,300.000000,5.000000,14977.990000


In [12]:
df.head()


,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39


In [14]:
df.dropna()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,TX002508,AC00297,856.21,2023-04-26 17:09:36,Credit,Colorado Springs,D000625,21.157.41.17,M072,Branch,33,Doctor,109,1,12690.79,2024-11-04 08:11:29
2508,TX002509,AC00322,251.54,2023-03-22 17:36:48,Debit,Tucson,D000410,49.174.157.140,M029,Branch,48,Doctor,177,1,254.75,2024-11-04 08:11:42
2509,TX002510,AC00095,28.63,2023-08-21 17:08:50,Debit,San Diego,D000095,58.1.27.124,M087,Branch,56,Retired,146,1,3382.91,2024-11-04 08:08:39
2510,TX002511,AC00118,185.97,2023-02-24 16:24:46,Debit,Denver,D000634,21.190.11.223,M041,Online,23,Student,19,1,1776.91,2024-11-04 08:12:22


In [17]:
# Format date properly
df['TransactionDate'] =  pd.to_datetime(df['TransactionDate'])
df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'])

# Calculate time since previous transcation in hours
df['TimeSincePrevTx'] = (df['TransactionDate'] - df['PreviousTransactionDate']).dt.total_seconds() / 3600

In [19]:
df['TransactionHour'] = df['TransactionDate'].dt.hour
df['TransactionDay'] = df['TransactionDate'].dt.day_name()
df['IsWeekend'] = df['TransactionDate'].dt.dayofweek >= 5

In [32]:
# Features
account_stats = df.groupby('AccountID').agg({
    'TransactionAmount': ['mean', 'std', 'max'],
    'TransactionDuration': ['mean', 'std'],
    'LoginAttempts': ['mean', 'max']
})
account_stats.columns = ['_'.join(col).strip() for col in account_stats.columns.values]
account_stats = account_stats.reset_index()
df = pd.merge(df, account_stats, on='AccountID', how='left')


In [34]:
df.head()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,...,TransactionDuration_std_y,LoginAttempts_mean_y,LoginAttempts_max_y,TransactionAmount_mean,TransactionAmount_std,TransactionAmount_max,TransactionDuration_mean,TransactionDuration_std,LoginAttempts_mean,LoginAttempts_max
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,...,73.461684,1.000,1,460.698571,487.969555,1397.59,138.428571,73.461684,1.000,1
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,...,59.159188,1.000,1,304.622857,240.297881,787.41,128.857143,59.159188,1.000,1
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,...,43.645351,1.000,1,237.047500,197.693348,489.31,71.250000,43.645351,1.000,1
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,...,61.836189,1.375,4,236.483750,275.157989,890.24,100.500000,61.836189,1.375,4
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,...,74.659226,1.000,1,280.796667,310.729033,826.23,119.000000,74.659226,1.000,1


In [35]:
df.columns

Index(['TransactionID', 'AccountID', 'TransactionAmount', 'TransactionDate',
       'TransactionType', 'Location', 'DeviceID', 'IP Address', 'MerchantID',
       'Channel', 'CustomerAge', 'CustomerOccupation', 'TransactionDuration',
       'LoginAttempts', 'AccountBalance', 'PreviousTransactionDate',
       'TimeSincePrviousTx', 'TimeSincePrevTx', 'TransactionHour',
       'TransactionDay', 'IsWeekend', 'TransactionAmount_mean_x',
       'TransactionAmount_std_x', 'TransactionAmount_max_x',
       'TransactionDuration_mean_x', 'TransactionDuration_std_x',
       'LoginAttempts_mean_x', 'LoginAttempts_max_x',
       'TransactionAmount_mean_y', 'TransactionAmount_std_y',
       'TransactionAmount_max_y', 'TransactionDuration_mean_y',
       'TransactionDuration_std_y', 'LoginAttempts_mean_y',
       'LoginAttempts_max_y', 'TransactionAmount_mean',
       'TransactionAmount_std', 'TransactionAmount_max',
       'TransactionDuration_mean', 'TransactionDuration_std',
       'LoginAttempts_m

In [36]:
# Calculate amount ratio compared to average
df['AmountToAvgRatio'] = df['TransactionAmount'] / df['TransactionAmount_mean']

In [37]:
df['AmountToAvgRatio']

0       0.030584
1       1.235101
2       0.532762
3       0.780180
4       0.047899
          ...   
2507    3.132021
2508    0.738214
2509    0.071983
2510    0.594818
2511    1.235062
Name: AmountToAvgRatio, Length: 2512, dtype: float64

In [41]:
df['UnusualLocation'] = ~df['Location'].isin(df.groupby('AccountID')['Location'].agg(lambda x: x.mode()[0]))
df['IPClass'] = df['IP Address'].apply(lambda x: ipaddress.ip_address(x).version)

In [42]:
numeric_features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 
                        'AccountBalance', 'CustomerAge', 'TimeSincePrevTx',
                        'TransactionAmount_mean', 'TransactionAmount_std', 'TransactionAmount_max',
                        'TransactionDuration_mean', 'TransactionDuration_std', 
                        'LoginAttempts_mean', 'LoginAttempts_max', 'AmountToAvgRatio']
    
categorical_features = ['TransactionType', 'Location', 'Channel', 
                           'CustomerOccupation', 'TransactionDay', 'IPClass']